# Convex Nonparametric Least Square (`CNLS`)

   + Author : Sheng Dai (sheng.dai@aalto.fi)
   + Date : April 26, 2020

References:

[1] Kuosmanen, T., Johnson, A. and Saastamoinen, A. (2015). Stochastic Nonparametric Approach to Efficiency Analysis: A unified Framework, in Zhu, J. (ed.) Data Envelopment Analysis. Springer, pp. 191–244.

[2] Johnson, A. L. and Kuosmanen, T. (2015). An Introduction to CNLS and StoNED Methods for Efficiency Analysis: Economic Insights and Computational Aspects, in Ray, S. C., Kumbhakar, S. C., and Dua, P. (eds) Benchmarking for Performance Evaluation: A Production Frontier Approach. Springer, pp. 117–186.

## Estimating production function

Hildreth (1954) was the first to consider nonparametric regression subject to monotonicity and concavity constraints in the case of a single input variable $x$. Kuosmanen (2008) extended Hildreth’s approach to the multivariate setting with a vector-valued $\bf{x}$, and coined the term convex nonparametric least squares (`CNLS`) for this method. `CNLS` builds upon the assumption that the true but unknown production function $f$ belongs to the set of continuous, monotonic increasing and globally concave functions, imposing exactly the same production axioms as standard DEA. 

The multivariate `CNLS` formulation is defined as:

\begin{align*}
& \underset{\alpha, \beta, \varepsilon} {min} \sum_{i=1}^n\varepsilon_i^2 \\
& \text{s.t.} \\
&  y_i = \alpha_i + \beta_i^{'}X_i + \varepsilon_i \quad \forall i \\
&  \alpha_i + \beta_i^{'}X_i \le \alpha_j + \beta_j^{'}X_i  \quad  \forall i, j\\
&  \beta_i \ge 0 \quad  \forall i \\
\end{align*}

where $\alpha_i$ and $\beta_i$ define the intercept and slope parameters of tangent hyperplanes that characterize the estimated piece-wise linear frontier. $\varepsilon_i$ denotes the CNLS residuals. The first constraint can be interpreted as a multivariate regression equation, the second constraint imposes convexity, and the third constraint imposes monotonicity.

## Example

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# import the package pystoned
from pystoned import CNLS

In [ ]:
# import Finnish electricity distribution firms data
url = 'https://raw.githubusercontent.com/ds2010/pyStoNED-Tutorials/master/Data/firms.csv'
df = pd.read_csv(url, error_bad_lines=False)

In [ ]:
# output
y = df['Energy']

# inputs
x1 = df['OPEX']
x1 = np.asmatrix(x1).T
x2 = df['CAPEX']
x2 = np.asmatrix(x2).T
x = np.concatenate((x1, x2), axis=1)

In [ ]:
# define and solve the CNLS model

cet = "addi"
fun = "prod"
rts = "vrs"

model = CNLS.cnls(y, x, cet, fun, rts)

# using local solver (MOSEK API)
from pyomo.opt import SolverFactory
opt = SolverFactory("mosek")
results = opt.solve(model, tee=True)

In [ ]:
# display the estimates (alpha, beta, and residual)
model.a.display()
model.b.display()
model.e.display()

In [ ]:
# retrive the alpha
val = list(model.a[:].value)
alpha = np.asarray(val)
alpha

In [ ]:
# retrive the residuals
val = list(model.e[:].value)
eps = np.asarray(val)
eps

In [ ]:
# retrive the beta
ind = list(model.b)
val = list(model.b[:, :].value)
beta = np.asarray([i + tuple([j]) for i, j in zip(ind, val)])

beta = pd.DataFrame(beta, columns=['Name', 'Key', 'Value'])
beta = beta.pivot(index='Name', columns='Key', values='Value')
beta.columns = ['b1', 'b2']
beta